In [1]:
import os, sys
import numpy as np
import torch
import torchaudio
import matplotlib.pyplot as plt
import IPython.display as ipd
from asteroid.metrics import get_metrics
from pprint import pprint
import time
from tqdm import tqdm

os.environ['CUDA_VISIBLE_DEVICES'] = '3'
torch.cuda.empty_cache()

# Get the pretrained models
print("Pre-trained models available:")
for model_name in os.listdir('../../pretrained_models'):
    print(model_name)
    
def normalize_tensor_wav(wav_tensor, eps=1e-8, std=None):
    mean = wav_tensor.mean(-1, keepdim=True)
    if std is None:
        std = wav_tensor.std(-1, keepdim=True)
    return (wav_tensor - mean) / (std + eps)
    
anechoic_model_p = '../../pretrained_models/GroupCom_Sudormrf_U8_Bases512_WSJ02mix.pt'
anechoic_model_p = '../../pretrained_models/Improved_Sudormrf_U16_Bases512_WSJ02mix.pt'
anechoic_model_p = '../../pretrained_models/Improved_Sudormrf_U36_Bases2048_WSJ02mix.pt'
noisy_reverberant_model_p = '../../pretrained_models/Improved_Sudormrf_U16_Bases2048_WHAMRexclmark.pt'
noisy_reverberant_model_p = '../../pretrained_models/Improved_Sudormrf_U36_Bases4096_WHAMRexclmark.pt'

# Load the appropriate class modules
sys.path.append("../../")
import sudo_rm_rf.dnn.models.improved_sudormrf as improved_sudormrf
import sudo_rm_rf.dnn.models.groupcomm_sudormrf_v2 as sudormrf_gc_v2
import sudo_rm_rf.dnn.models.sepformer as sepformer
from speechbrain.pretrained import SepformerSeparation as sep_former_separator


Pre-trained models available:
Improved_Sudormrf_U16_Bases2048_WHAMRexclmark.pt
improved_sudo_epoch_500
GroupCom_Sudormrf_U8_Bases512_WSJ02mix.pt
Improved_Sudormrf_U16_Bases512_WSJ02mix.pt
Improved_Sudormrf_U36_Bases4096_WHAMRexclmark.pt
Improved_Sudormrf_U36_Bases2048_WSJ02mix.pt


In [2]:
# Compare all models for forward and backward runs on GPU
def count_parameters(model):
    numparams = 0
    for f in model.parameters():
        if f.requires_grad:
            numparams += f.numel()
    print('Trainable Parameters (millions): {}'.format(
        round(numparams / 10**6, 3)))
    return numparams

def backward_pass(model, input_samples=32000,
                  repeats=1, bs=4, n_sources=2, mode='cpu', is_sudo_model=True):
    opt = torch.optim.Adam(model.parameters(), lr=0.001)
    def l1_wrapper(x, y):
        return torch.mean(torch.abs(x - y))
    tr_loss = l1_wrapper
    
    if is_sudo_model:
        mixture = torch.rand([bs, 1, input_samples])
        clean_wavs = torch.rand([bs, n_sources, input_samples])
    else:
        mixture = torch.rand([bs, input_samples])
        clean_wavs = torch.rand([bs, input_samples, n_sources])
    
    if mode == 'gpu':
        model = model.cuda()
        mixture = mixture.cuda()
        clean_wavs = clean_wavs.cuda()
    model.train()
    
    for f in model.parameters():
        f.requires_grad = True
    
    count_parameters(model)

    total_time = 0.
    for i in range(repeats):
        now = time.time()
        opt.zero_grad()
        est_sources = model(mixture)
        l = tr_loss(est_sources, clean_wavs)
        l.backward()
        opt.step()
        total_time += time.time() - now
    avg_time = (total_time) / repeats
    peak_cuda_mem = torch.cuda.max_memory_allocated() / 10 ** 9
    print(f'Elapsed Time Backward DEV: {mode} BS: {bs}, {avg_time} sec, Peak GPU usage {peak_cuda_mem} GB')

    
    if mode == 'gpu':
        del model, opt, mixture, clean_wavs
    
    return avg_time, peak_cuda_mem
    
    
def forward_pass(model, input_samples=32000,
                  repeats=1, bs=4, n_sources=2, mode='cpu', is_sudo_model=True):    
    if is_sudo_model:
        mixture = torch.rand([bs, 1, input_samples])
    else:
        mixture = torch.rand([bs, input_samples])
    
    if mode == 'gpu':
        model = model.cuda()
        mixture = mixture.cuda()

    total_time = 0.
    model.eval()
    with torch.no_grad():
        for i in range(repeats):
            now = time.time()
            est_sources = model(mixture)
            total_time += time.time() - now
        avg_time = (total_time) / repeats
    peak_cuda_mem = torch.cuda.max_memory_allocated() / 10 ** 9
    print(f'Elapsed Time Forward DEV: {mode} BS: {bs}, {avg_time} sec, Peak GPU usage {peak_cuda_mem} GB')
    
    if mode == 'gpu':
        del model, mixture
    
    return avg_time, peak_cuda_mem
    

In [3]:
models_to_eval = [
#     {
#         'model_path': '../../pretrained_models/GroupCom_Sudormrf_U8_Bases512_WSJ02mix.pt',
#         'is_sudo_model': True,
#         'bs_list': [1, 2, 4, 6]
#     },
#     {
#         'model_path': '../../pretrained_models/Improved_Sudormrf_U16_Bases512_WSJ02mix.pt',
#         'is_sudo_model': True,
#         'bs_list': [1, 2, 4]
#     },
    {
        'model_path': '../../pretrained_models/Improved_Sudormrf_U36_Bases2048_WSJ02mix.pt',
        'is_sudo_model': True,
        'bs_list': [1]
    },
#     {
#         'model_path': '../../pretrained_models/Improved_Sudormrf_U16_Bases2048_WHAMRexclmark.pt',
#         'is_sudo_model': True,
#         'bs_list': [1, 2, 4]
#     },
#     {
#         'model_path': '../../pretrained_models/Improved_Sudormrf_U36_Bases4096_WHAMRexclmark.pt',
#         'is_sudo_model': True,
#         'bs_list': [1]
#     },
#     {
#         'model_path': None,
#         'is_sudo_model': False,
#         'bs_list': [1]
#     }
]

def get_model(model_info, is_gpu=False):
    if model_info['model_path'] is None:
        if is_gpu:
            model = sep_former_separator.from_hparams(source="speechbrain/sepformer-wsj02mix", 
                                           savedir='pretrained_models/sepformer-wsj02mix',
                                           run_opts={"device":"cuda"})
        else:
            model = sep_former_separator.from_hparams(source="speechbrain/sepformer-wsj02mix", 
                                           savedir='pretrained_models/sepformer-wsj02mix')
        
        model_name = "Sepformer"
    else:
        model = torch.load(model_info['model_path'])
        model_name = model_info['model_path'].split('/')[-1]
    return model, model_name


for model_info in models_to_eval:
    model, model_name = get_model(model_info, is_gpu=False)
    model.train()
    print("======================")
    print(f"Evaluating model: {model_name}")
    
    for bs in model_info['bs_list']:
        
        model, model_name = get_model(model_info)
        avg_time, cuda_max_mem = forward_pass(model, input_samples=32000,
                  repeats=10, bs=bs, n_sources=2, mode='cpu', is_sudo_model=model_info['is_sudo_model'])
        
        torch.cuda.empty_cache()
        
        model, model_name = get_model(model_info, is_gpu=True)
        avg_time, cuda_max_mem = forward_pass(model, input_samples=32000,
                  repeats=20, bs=bs, n_sources=2, mode='gpu', is_sudo_model=model_info['is_sudo_model'])

        torch.cuda.empty_cache()
        
        model, model_name = get_model(model_info, is_gpu=True)
        avg_time, cuda_max_mem = backward_pass(model, input_samples=32000,
                  repeats=20, bs=bs, n_sources=2, mode='gpu', is_sudo_model=model_info['is_sudo_model'])
        
        torch.cuda.empty_cache()
        
        

Evaluating model: Improved_Sudormrf_U36_Bases2048_WSJ02mix.pt
Elapsed Time Forward DEV: cpu BS: 1, 3.1384077787399294 sec, Peak GPU usage 0.0 GB
Elapsed Time Forward DEV: gpu BS: 1, 0.09933750629425049 sec, Peak GPU usage 0.229693952 GB
Trainable Parameters (millions): 23.239
Elapsed Time Backward DEV: gpu BS: 1, 0.4485529184341431 sec, Peak GPU usage 5.262242304 GB


In [30]:
computation_metrics_results = """
======================
Evaluating model: GroupCom_Sudormrf_U8_Bases512_WSJ02mix.pt
Elapsed Time Forward DEV: cpu BS: 1, 0.6553239345550537 sec, Peak GPU usage 0.0 GB
Elapsed Time Forward DEV: gpu BS: 1, 0.02278091907501221 sec, Peak GPU usage 0.061922816 GB
Elapsed Time Forward DEV: cpu BS: 2, 1.4939395904541015 sec, Peak GPU usage 0.061922816 GB
Elapsed Time Forward DEV: gpu BS: 2, 0.02694549560546875 sec, Peak GPU usage 0.121698816 GB
Elapsed Time Forward DEV: cpu BS: 4, 3.5484800577163695 sec, Peak GPU usage 0.121698816 GB
Elapsed Time Forward DEV: gpu BS: 4, 0.05067609548568726 sec, Peak GPU usage 0.246493696 GB
Elapsed Time Forward DEV: cpu BS: 6, 6.017881846427917 sec, Peak GPU usage 0.246493696 GB
Elapsed Time Forward DEV: gpu BS: 6, 0.07618927955627441 sec, Peak GPU usage 0.362899968 GB

Trainable Parameters (millions): 0.507
Elapsed Time Backward DEV: gpu BS: 1, 0.12531145811080932 sec, Peak GPU usage 1.451049472 GB
Trainable Parameters (millions): 0.507
Elapsed Time Backward DEV: gpu BS: 2, 0.14525175094604492 sec, Peak GPU usage 2.905540096 GB
Trainable Parameters (millions): 0.507
Elapsed Time Backward DEV: gpu BS: 4, 0.2209705352783203 sec, Peak GPU usage 5.941104128 GB
Trainable Parameters (millions): 0.507
Elapsed Time Backward DEV: gpu BS: 6, 0.3077015161514282 sec, Peak GPU usage 8.738743808 GB

======================
Evaluating model: Improved_Sudormrf_U16_Bases512_WSJ02mix.pt
Elapsed Time Forward DEV: cpu BS: 1, 1.0378467798233033 sec, Peak GPU usage 0.0 GB
Elapsed Time Forward DEV: gpu BS: 1, 0.03812741041183472 sec, Peak GPU usage 0.0765824 GB
Elapsed Time Forward DEV: cpu BS: 2, 2.2890385150909425 sec, Peak GPU usage 0.0765824 GB
Elapsed Time Forward DEV: gpu BS: 2, 0.04016602039337158 sec, Peak GPU usage 0.1330816 GB
Elapsed Time Forward DEV: cpu BS: 4, 4.936136507987976 sec, Peak GPU usage 0.1330816 GB
Elapsed Time Forward DEV: gpu BS: 4, 0.07500633001327514 sec, Peak GPU usage 0.254468608 GB

Trainable Parameters (millions): 5.016
Elapsed Time Backward DEV: gpu BS: 1, 0.18350332975387573 sec, Peak GPU usage 2.099536384 GB
Trainable Parameters (millions): 5.016
Elapsed Time Backward DEV: gpu BS: 2, 0.21717438697814942 sec, Peak GPU usage 4.127073792 GB
Trainable Parameters (millions): 5.016
Elapsed Time Backward DEV: gpu BS: 4, 0.34041128158569334 sec, Peak GPU usage 8.43066112 GB


======================
Evaluating model: Improved_Sudormrf_U16_Bases2048_WHAMRexclmark.pt
Elapsed Time Forward DEV: cpu BS: 1, 1.2158073425292968 sec, Peak GPU usage 0.0 GB
Elapsed Time Forward DEV: gpu BS: 1, 0.03812267780303955 sec, Peak GPU usage 0.162170368 GB
Elapsed Time Forward DEV: cpu BS: 2, 2.798716926574707 sec, Peak GPU usage 0.162170368 GB
Elapsed Time Forward DEV: gpu BS: 2, 0.043878686428070066 sec, Peak GPU usage 0.288383488 GB
Elapsed Time Forward DEV: cpu BS: 4, 5.887118864059448 sec, Peak GPU usage 0.288383488 GB
Elapsed Time Forward DEV: gpu BS: 4, 0.0821913480758667 sec, Peak GPU usage 0.551295488 GB

Trainable Parameters (millions): 6.363
Elapsed Time Backward DEV: gpu BS: 1, 0.18807061910629272 sec, Peak GPU usage 2.443395584 GB
Trainable Parameters (millions): 6.363
Elapsed Time Backward DEV: gpu BS: 2, 0.21935930252075195 sec, Peak GPU usage 4.771350016 GB
Trainable Parameters (millions): 6.363
Elapsed Time Backward DEV: gpu BS: 4, 0.35598028898239137 sec, Peak GPU usage 9.697005056 GB

======================
Evaluating model: Improved_Sudormrf_U36_Bases2048_WSJ02mix.pt
Elapsed Time Forward DEV: cpu BS: 1, 3.144366216659546 sec, Peak GPU usage 0.0 GB
Elapsed Time Forward DEV: gpu BS: 1, 0.1018365502357483 sec, Peak GPU usage 0.229693952 GB
Trainable Parameters (millions): 23.239
Elapsed Time Backward DEV: gpu BS: 1, 0.46427216529846194 sec, Peak GPU usage 5.262242304 GB

======================
Evaluating model: Improved_Sudormrf_U36_Bases4096_WHAMRexclmark.pt
Elapsed Time Forward DEV: cpu BS: 1, 3.7126335859298707 sec, Peak GPU usage 0.0 GB
Elapsed Time Forward DEV: gpu BS: 1, 0.09852558374404907 sec, Peak GPU usage 0.368998912 GB
Elapsed Time Forward DEV: cpu BS: 2, 7.979483270645142 sec, Peak GPU usage 0.0 GB
Elapsed Time Forward DEV: gpu BS: 2, 0.11349769830703735 sec, Peak GPU usage 0.631526912 GB
Trainable Parameters (millions): 26.608
Elapsed Time Backward DEV: gpu BS: 1, 0.45119341611862185 sec, Peak GPU usage 5.727252992 GB

======================
Evaluating model: Sepformer
Elapsed Time Forward DEV: cpu BS: 1, 9.050826048851013 sec, Peak GPU usage 0.0 GB
Elapsed Time Forward DEV: gpu BS: 1, 0.09447981119155884 sec, Peak GPU usage 0.392123904 GB
Trainable Parameters (millions): 25.679
Elapsed Time Backward DEV: gpu BS: 1, 0.28271385431289675 sec, Peak GPU usage 8.156322816 GB

"""

In [44]:
# PArse the results
from pprint import pprint

comp_res_dic = {}
for model_res in computation_metrics_results.split("======================"):
    this_model = None
    for l in model_res.split("\n"):
        if l.startswith("Evaluating"):
            comp_res_dic[l.split(" ")[-1]] = {}
            this_model = l.split(" ")[-1]
            continue
        if l.startswith("Trainable"):
            comp_res_dic[this_model]["Params"] = str(round(float(l.split(" ")[-1]), 2))
        else:
            if "BS: 2" in l or "BS: 6" in l:
                continue
            
            info = l.split(" ")
            if "Backward" in l:
                if "mem gpu back" not in comp_res_dic[this_model]:
                    comp_res_dic[this_model]["mem gpu back"] = str(round(float(info[-2]), 2))
                else:
                    comp_res_dic[this_model]["mem gpu back"] += f" <br> {round(float(info[-2]), 2)}"
                if "time gpu back" not in comp_res_dic[this_model]:
                    comp_res_dic[this_model]["time gpu back"] = str(round(float(info[-7]), 2))
                else:
                    comp_res_dic[this_model]["time gpu back"] += f" <br> {round(float(info[-7]), 2)}"
                if "exsec gpu back" not in comp_res_dic[this_model]:
                    comp_res_dic[this_model]["exsec gpu back"] = str(round(this_bs / float(info[-7]), 1))
                else:
                    comp_res_dic[this_model]["exsec gpu back"] += f" <br> {round(this_bs / float(info[-7]), 1)}"
            elif "Forward DEV: gpu" in l:
                this_bs = int(info[-8][0])
                if "mem gpu for" not in comp_res_dic[this_model]:
                    comp_res_dic[this_model]["mem gpu for"] = str(round(float(info[-2]), 2))
                else:
                    comp_res_dic[this_model]["mem gpu for"] += f" <br> {round(float(info[-2]), 2)}"
                if "time gpu for" not in comp_res_dic[this_model]:
                    comp_res_dic[this_model]["time gpu for"] = str(round(float(info[-7]), 2))
                else:
                    comp_res_dic[this_model]["time gpu for"] += f" <br> {round(float(info[-7]), 2)}"
                if "exsec gpu for" not in comp_res_dic[this_model]:
                    comp_res_dic[this_model]["exsec gpu for"] = str(round(this_bs / float(info[-7]), 1))
                else:
                    comp_res_dic[this_model]["exsec gpu for"] += f" <br> {round(this_bs / float(info[-7]), 1)}"

            elif "Forward DEV: cpu" in l:
                if "time cpu for" not in comp_res_dic[this_model]:
                    comp_res_dic[this_model]["time cpu for"] = str(round(float(info[-7]), 2))
                else:
                    comp_res_dic[this_model]["time cpu for"] += f" <br> {round(float(info[-7]), 2)}"
                if "exsec cpu for" not in comp_res_dic[this_model]:
                    comp_res_dic[this_model]["exsec cpu for"] = str(round(this_bs / float(info[-7]), 1))
                else:
                    comp_res_dic[this_model]["exsec cpu for"] += f" <br> {round(this_bs / float(info[-7]), 1)}"
            print
for m in comp_res_dic:
    print("=========\n", m)
    pprint(comp_res_dic[m])
    acc = []
    for metric in ['exsec cpu for', 
                   'exsec gpu for', 'mem gpu for', 
                   'exsec gpu back', 'mem gpu back', 'Params']:
        acc.append(comp_res_dic[m][metric])

#     print(acc)
    print(" | ".join(acc))

 GroupCom_Sudormrf_U8_Bases512_WSJ02mix.pt
{'Params': '0.51',
 'exsec cpu for': '1.5 <br> 0.3',
 'exsec gpu back': '31.9 <br> 18.1',
 'exsec gpu for': '43.9 <br> 78.9',
 'mem gpu back': '1.45 <br> 5.94',
 'mem gpu for': '0.06 <br> 0.25',
 'time cpu for': '0.66 <br> 3.55',
 'time gpu back': '0.13 <br> 0.22',
 'time gpu for': '0.02 <br> 0.05'}
1.5 <br> 0.3 | 43.9 <br> 78.9 | 0.06 <br> 0.25 | 31.9 <br> 18.1 | 1.45 <br> 5.94 | 0.51
 Improved_Sudormrf_U16_Bases512_WSJ02mix.pt
{'Params': '5.02',
 'exsec cpu for': '3.9 <br> 0.2',
 'exsec gpu back': '21.8 <br> 11.8',
 'exsec gpu for': '26.2 <br> 53.3',
 'mem gpu back': '2.1 <br> 8.43',
 'mem gpu for': '0.08 <br> 0.25',
 'time cpu for': '1.04 <br> 4.94',
 'time gpu back': '0.18 <br> 0.34',
 'time gpu for': '0.04 <br> 0.08'}
3.9 <br> 0.2 | 26.2 <br> 53.3 | 0.08 <br> 0.25 | 21.8 <br> 11.8 | 2.1 <br> 8.43 | 5.02
 Improved_Sudormrf_U16_Bases2048_WHAMRexclmark.pt
{'Params': '6.36',
 'exsec cpu for': '3.3 <br> 0.2',
 'exsec gpu back': '21.3 <br> 11.2